In [1]:
import sympy as sp

# Define the symbols
alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff, S_x, S_y, S_z = sp.symbols(
    'alpha beta gama X Y Z L v1 v2 v3 n1 n2 n3 ff S_x S_y S_z', real=True)

# Sip and TCP
Sip = sp.Matrix([S_x, S_y, S_z])
TCP = sp.Matrix([X, Y, Z])

# Rotation matrices Rx, Ry, Rz
Rx = sp.Matrix([[1, 0, 0], [0, sp.cos(alpha), sp.sin(alpha)], [0, -sp.sin(alpha), sp.cos(alpha)]])
Ry = sp.Matrix([[sp.cos(beta), 0, -sp.sin(beta)], [0, 1, 0], [sp.sin(beta), 0, sp.cos(beta)]])
Rz = sp.Matrix([[sp.cos(gama), sp.sin(gama), 0], [-sp.sin(gama), sp.cos(gama), 0], [0, 0, 1]])
R = sp.simplify(Rx * Ry * Rz)

# Vector v and calculations for x and o
v = sp.Matrix([v1, v2, v3])
x = sp.Matrix([X, Y, Z]) - L * R * v
o = R * v

# Calculate 'a' using dot products
a = sp.simplify(sp.sqrt(((-R.row(0).dot(v) * Z) / (R.row(2).dot(v)))**2 +
                        (-(R.row(1).dot(v) * Z) / (R.row(2).dot(v)))**2 + Z * Z))

# Calculate Q
Q = a * o + TCP

# Project Sip onto the line defined by o and Q
Sip_proj = Sip + (o.dot(Q) - o.dot(Sip)) * (x - Sip) / (o.dot(x) - o.dot(Sip))

# s_2 vector and s_1 as the cross product of o and s_2
s_2 = R @ sp.Matrix([n1, n2, n3])
s_1 = o.cross(s_2)

# Calculate k_1 and k_2
k_1 = (Sip_proj - Q).dot(s_1) / s_1.norm()**2
k_2 = (Sip_proj - Q).dot(s_2) / s_2.norm()**2

# Q_t as a zero vector (for 2D image plane coordinates)
Q_t = sp.Matrix([0, 0])

# Calculate image plane coordinates f_1 and f_2
f_1 = Q_t[0] + k_1 * ff / (a + L - ff)
f_2 = Q_t[1] + k_2 * ff / (a + L - ff)

# Define the symbols for S
S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15 = sp.symbols("S1:16", real=True)
M = sp.Matrix(5, 3, (S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15))

# Define the matrix to store the results of f1 and f2 for all S
results = sp.Matrix(10, 1, [0] * 10)

for i in range(5):
    # Substitute the symbols from M for S_x, S_y, S_z in f_1 and f_2
    f1_i = f_1.subs({S_x: M[i, 0], S_y: M[i, 1], S_z: M[i, 2]})
    f2_i = f_2.subs({S_x: M[i, 0], S_y: M[i, 1], S_z: M[i, 2]})
    # Store the results in the matrix (also convert to mm)
    results[2 * i, 0] = f1_i * 1e3
    results[2 * i + 1, 0] = f2_i * 1e3

# observations for minsquare    
f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10 = sp.symbols('f_obs1:11', real=True)
f_obs = sp.Matrix(10, 1, (f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10))

# MinSquare functional
diff = results - f_obs
F_obs = diff.norm()**2

In [4]:
import inspect

args = [alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff, 
        S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15,
        f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10]
F_obs_lambd = sp.lambdify(args, F_obs, modules='jax', cse=True)

function_text = inspect.getsource(F_obs_lambd)


def _lambdifygenerated(alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff, S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15, f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10):
    x0 = sin(beta)
    x1 = cos(gama)
    x2 = cos(beta)
    x3 = x1*x2
    x4 = sin(gama)
    x5 = x2*x4
    x6 = n1*x3 + n2*x5 - n3*x0
    x7 = v3*x0
    x8 = v1*x3
    x9 = v2*x5
    x10 = -L*x7 + L*x8 + L*x9 - X
    x11 = sin(alpha)
    x12 = x1*x11
    x13 = cos(alpha)
    x14 = x13*x4
    x15 = -x0*x14 + x12
    x16 = -v2*x15
    x17 = x11*x4
    x18 = x1*x13
    x19 = x0*x18 + x17
    x20 = v1*x19
    x21 = x13*x2
    x22 = v3*x21
    x23 = x20 + x22
    x24 = x16 + x23
    x25 = L*x16 + L*x20 + L*x22 - Z
    x26 = x11*x2
    x27 = v3*x26
    x28 = x0*x17 + x18
    x29 = v2*x28
    x30 = x0*x12 - x14
    x31 = v1*x30
    x32 = x27 + x29 + x31
    x33 = L*x27 + L*x29 + L*x31 - Y
    x34 = -x7 + x8 + x9
    x35 = x10*x34 + x24*x25 + x32*x33
    x36 = S13*